### Generación de un dataset de test, 1 ejemplo de cada triada de categorias.

#### Un dataframe para productos y otro para texto a procesar por el modelo de embeddings.

In [1]:
import json
import pandas as pd
import uuid

# Leer el archivo Excel
df = pd.read_excel('./data/data.xlsx', engine='openpyxl')

# Definir la ruta de salida
out_path = './data/utils/'

# Guardar el dataframe como JSON con 4 espacios de indentación y codificación utf-8
df.to_json(out_path+'data.json', orient='records', indent=4, force_ascii=False)

# Guardar el dataframe como CSV separado por '*' y sin índices
df.to_csv(out_path+'data.csv', sep='*', index=False)

# Obtener un nuevo dataframe con el primer ejemplo de cada triada de categorías
unique_df = df.drop_duplicates(subset=['Categoria 1', 'Categoria 2', 'Categoria 3'], keep='first').reset_index(drop=True)

# Guardar el nuevo dataframe en un archivo JSON bien formateado con codificación utf-8
columns = unique_df.columns
aux = []

# Para cada fila del dataframe, crear un diccionario y agregarlo a la lista 'aux'
for row in unique_df.itertuples():
    item = {}
    for i in range(0, len(columns)):
        item[columns[i]] = row[i+1]
    aux.append({
        "input_text": item,
        "output_text": ""
    })

# Guardar la lista 'aux' como JSON en un archivo
with open(out_path+'finetune-test.json', 'w', encoding='utf-8') as f:
    json.dump(aux, f, ensure_ascii=False, indent=4)

# Guardar el nuevo dataframe como CSV separado por '*'
unique_df.to_csv(out_path+'finetune-test.csv', sep='*', index=False) # DATA PARA FINETUNING (No se usa)

# Agregar UUIDs a cada ejemplo en el dataframe
test_df = unique_df.copy()
test_df.insert(0, 'uuid', [uuid.uuid4() for _ in range(len(test_df))])
test_df = test_df.sort_values(by=['uuid']).reset_index(drop=True)

# Guardar el dataframe con UUIDs como CSV separado por '*' y con codificación utf-8
test_df.to_csv(out_path+'products-test.csv', sep='*', index=False, encoding='utf-8') # DATA PARA TESTEAR

# Leer el dataframe con UUIDs desde el archivo CSV
test_embed_df = pd.read_csv(out_path+'products-test.csv', sep='*', encoding='utf-8')

# Crear un nuevo dataframe para hacer embeddings con columnas 'uuid' y 'text_content'
embed_df = pd.DataFrame()
embed_df['uuid'] = test_embed_df['uuid']
embed_df['text_content'] = ''

# Para cada UUID, construir el texto para 'text_content'
for uuid in embed_df['uuid']:
    text_content = ''
    for col in ['Marca', 'Nombre', 'Descripción del Producto', 'Badge Promocional']:
        text_content += col + ": " + str(test_embed_df.loc[test_embed_df['uuid'] == uuid][col].values[0]) + '\n'
        
    text_content += 'Categorias: '
    for col in ['Categoria 1', 'Categoria 2', 'Categoria 3']:
        if not pd.isnull(test_embed_df.loc[test_embed_df['uuid'] == uuid][col].values[0]):
            text_content += str(test_embed_df.loc[test_embed_df['uuid'] == uuid][col].values[0]) + ' >> '
    
    text_content = text_content[:-3]  # Quitar el último '>>' agregado
    embed_df.loc[embed_df['uuid'] == uuid, 'text_content'] = text_content

# Convertir 'text_content' a tipo string y verificar nulos
embed_df['text_content'] = embed_df['text_content'].astype(str)
embed_df.isnull().sum()

# Mostrar el texto del primer ejemplo y guardar el dataframe como CSV
example = embed_df.loc[0, 'text_content']
embed_df.to_csv(out_path+'embedding-test.csv', sep='*', index=False, encoding='utf-8') # DATA PARA CALCULAR EMBEDDINGS
print(f"Se guardó el dataframe con embeddings en {out_path+'embedding-test.csv'}") 


Se guardó el dataframe con embeddings en ./data/utils/embedding-test.csv


In [8]:
embed_df.tail() # Textos para calcular embeddings

,uuid,text_content
82,ebefee88-5fa4-4954-bf7c-2cc21e2e2c77,Marca: Samsung\nNombre: Horno de Microondas 1....
83,f1bb12a7-a190-435f-b817-133c912305b7,Marca: Bazhars\nNombre: Tapete Decorativo Flow...
84,f8959271-9cbe-4872-9693-57527f55942d,Marca: True\nNombre: Carrito de Servicio para ...
85,fcc00a8c-cfac-4ad9-a7e4-04d2b6e03e4a,Marca: Duartee&Co\nNombre: Caja Decorativa en ...
86,fde7d90d-c7f4-467b-bee5-352e9cf8e392,Marca: Violanti\nNombre: Sala Design Gris Oxfo...


In [7]:
unique_df.tail() # DATA PARA FINETUNING (No se usa)

,Enlace,Marca,Nombre,Badge Promocional,Descripción del Producto,Precio Anterior,Precio Actual,Categoria 1,Categoria 2,Categoria 3
82,https://www.elpalaciodehierro.com/samsonite-ma...,Samsonite,Maleta de viaje TOIIS C Negra,Meses Sin Intereses,Maleta de viaje Samsonite TOIIS C elaborada en...,6599.0,4619.30,mochilas-y-maletas-de-viaje,NaN,NaN
83,https://www.elpalaciodehierro.com/assouline-pa...,Assouline,Paris Chic,NaN,"Oliver Pilcher, ""Paris Chic""; París es la ciud...",2950.0,2507.50,libros,NaN,NaN
84,https://www.elpalaciodehierro.com/royal-canin-...,Royal Canin,Alimento Seco para Perro Cachorro Razas Pequeñ...,NaN,Alimento Seco para Perro Cachorro Razas Pequeñ...,405.0,364.50,mascotas,NaN,NaN
85,https://www.elpalaciodehierro.com/violanti-sal...,Violanti,Sala Design Gris Oxford,Meses Sin Intereses,Sala Violanti Design elaborada en madera tapiz...,42999.0,25799.40,outlet,hogar,NaN
86,https://www.elpalaciodehierro.com/urban-republ...,Urban Republic Life Styles,Pie de árbol Villa en tela,Solo en El Palacio,Pie de árbol Urban Republic Life Styles de la ...,2359.0,1769.25,temporada,navidad,decoracion-navidena


In [5]:
test_df.tail() # Con uuids y dejando solo un ejemplo de cada triada de categorías

,uuid,Enlace,Marca,Nombre,Badge Promocional,Descripción del Producto,Precio Anterior,Precio Actual,Categoria 1,Categoria 2,Categoria 3
82,ebefee88-5fa4-4954-bf7c-2cc21e2e2c77,https://www.elpalaciodehierro.com/samsung-horn...,Samsung,Horno de Microondas 1.1 pies AME8114ST/XAX Negro,Meses Sin Intereses,Horno de Microondas Samsung 1.1 pies Negro; de...,4299.0,2579.4,hogar,electrodomesticos,horno-microondas
83,f1bb12a7-a190-435f-b817-133c912305b7,https://www.elpalaciodehierro.com/bazhars-tape...,Bazhars,"Tapete Decorativo Flow Negro, Beige 230x160cm",NaN,Tapete Decorativo Bazhars Flow elaborado en ma...,7530.0,5271.0,hogar,decoracion,tapetes
84,f8959271-9cbe-4872-9693-57527f55942d,https://www.elpalaciodehierro.com/true-carrito...,True,Carrito de Servicio para Bar Dorado,Meses Sin Intereses,Carrito de Servicio para Bar True Dorado; Su s...,11975.0,9580.0,hogar,cocina,almacenamiento-organizacion
85,fcc00a8c-cfac-4ad9-a7e4-04d2b6e03e4a,https://www.elpalaciodehierro.com/duarteeco-ca...,Duartee&Co,Caja Decorativa en cartón,Meses Sin Intereses,Caja decorativa Duartee&Co elaborada en materi...,790.0,553.0,hogar,organizacion-limpieza,cajas-y-contenedores
86,fde7d90d-c7f4-467b-bee5-352e9cf8e392,https://www.elpalaciodehierro.com/violanti-sal...,Violanti,Sala Design Gris Oxford,Meses Sin Intereses,Sala Violanti Design elaborada en madera tapiz...,42999.0,25799.4,hogar,muebles,salas
